In [3]:
from queue import PriorityQueue
from printTree import display_tree

import networkx as nx 
from networkx.drawing.nx_agraph import write_dot, graphviz_layout #TODO
import matplotlib.pyplot as plt 

In [86]:
class Huffman_tree:
    class Node:
        def __init__(self, letter, freq, left = None, right = None):
            self.letter = letter
            self.freq = freq
            self.left = left
            self.right = right
            
            self.code = ''
            self.symbol = ''
        
        def __lt__(self, other):
            return self.freq < other.freq
        
    def __init__(self, text):
        self.freq = {}
        
        for l in text:
            if l in self.freq:
                self.freq[l] += 1
            else:
                self.freq[l] = 1
        print(self.freq)
        q = PriorityQueue()
        
        for l, f in self.freq.items():
            q.put((f, self.Node(l, f)))
        
        while q.qsize() > 1:
            _, left = q.get()
            _, right = q.get()
            
            
            left.symbol = '0'
            right.symbol = '1'
            
            new_node = self.Node(None, left.freq + right.freq, left, right)
            
            q.put((new_node.freq, new_node))

        _, self.root = q.get()
        
        self.code = {}
        
        self.get_code(self.root, "")
    
    
    def get_code(self, node, prev_code):
        node.code = prev_code + node.symbol
        
        if node.letter is not None:
            self.code[node.letter] = node.code
        
        if node.left is not None:
            self.get_code(node.left, node.code)
        
        if node.right is not None:
            self.get_code(node.right, node.code)
    
    def print(self):
        def _to_print(root):
            if root.letter is not None:
                return " " + str(root.letter) + ":" + root.code
            return "_"
        display_tree(self.root, whatToPrint=_to_print)
    
    

In [87]:
text = """abaaabcbsbcbjdssbbbaababa"""

huff = Huffman_tree(text)

{'a': 8, 'b': 10, 'c': 2, 's': 3, 'j': 1, 'd': 1}


In [88]:
huff.print()


   __________________________________      
  /                                  \     
 b:0        ____________________________   
           /                            \  
         __________                    a:11
        /          \                       
      s:100     ____________               
               /            \              
             c:1010      ________          
                        /        \         
                     j:10110  d:10111      



In [89]:
print(huff.code)

{'b': '0', 's': '100', 'c': '1010', 'j': '10110', 'd': '10111', 'a': '11'}


In [123]:
def text_to_code(text, code):
    res = ""
    
    for l in text:
        res += code[l]
    
    return res

In [142]:
def text_from_code(code, tree):
    res = ""
    tmp = ""
    node = tree.root
    for c in code:  
        if c == '0':
            node = node.left
        else:
            node = node.right
        
        if node.left is None and node.right is None:
            res += node.letter
            node = tree.root
        
    return res

In [143]:
coded = text_to_code(text, huff.code)
print(coded)

11011111101010010001010010110101111001000001111011011


In [144]:
decoded = text_from_code(coded, huff)
print(decoded)
print(text)

abaaabcbsbcbjdssbbbaababa
abaaabcbsbcbjdssbbbaababa
